# web scraping RIP.ie with beautifulsoup

In [21]:
import time
import pandas as pd
from bs4 import BeautifulSoup
import requests
import datefinder

def get_dn_page(n):
    """Get death notice text from page matching the id number"""
    
    url = 'https://rip.ie/showdn.php?dn=%s' %n
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    if soup.title == None:
        title = ''
    else:        
        title  = soup.title.text.strip()
    name=''    
    for s in ['Death Notice of ','The death has occurred of ']:
        if title.startswith(s):
            name = title.split(s)[1]   
    elem = soup.find_all("div", id="dn_photo_and_text")
    
    if len(elem) == 0:
        return name, '', '', '', ''
    rows = elem[0].find_all('p')
    if len(rows) == 0:
        rows = elem[0].find_all('td')
    text = ';'.join([r.text.strip() for r in rows]).replace('\n','')
    #address
    addrelem = soup.find("span", class_='small_addr') 
    if addrelem != None:
        address = addrelem.text.strip()
    else:
        address = ''
    #county  
    ctyelem = soup.find("li", class_='fd_county') 
    if ctyelem != None:
        county = ctyelem.text.strip()
    else:
        county = ''
    #date
    dateelem = soup.find("div", class_='ddeath')
    if dateelem == None:
        dateelem = soup.find("div", class_='dpubl')
    s = dateelem.text.strip()
    try:
        date = list(datefinder.find_dates(s))[0]
    except:
        date = ''
    print (n, date, name, address, county)
    return name, date, county, address, text

In [2]:
get_dn_page(390045)

390045 2019-06-05 00:00:00 Margaret  Kelly Avenue Grove, Ballymodan Place, Bandon,  Cork Cork


('Margaret  Kelly',
 datetime.datetime(2019, 6, 5, 0, 0),
 'Cork',
 'Avenue Grove, Ballymodan Place, Bandon,  Cork',
 "Kelly (Avenue Grove, Ballymodan Place, Bandon and late of Currivreeda West) on June 5th 2019. Margaret, beloved daughter of the late Timothy and Ellen. Sadly missed by her loving sisters Ann and Eileen, brothers Teddy and John, Margaret's Partner William, brothers-in-law, sisters-in-law, nieces, nephews, relatives and good friend Malcolm.;;Rosary on Friday evening at 7pm in St. Patrick's Church, Bandon. Requiem Mass on Saturday at 12 noon, funeral afterwards to the adjoining cemetery.;;May Margaret Rest in Peace")

In [3]:
#df = pd.read_csv('rip_dn_scrape.csv')
df = pd.read_pickle('rip_dn_scrape.pkl')
len(df)

484851

## iterate over a range of ids and get info

In [22]:
#read current table in so we skip those already done
df = pd.read_pickle('rip_dn_scrape.pkl')
print (len(df))
ids = list(df.id)

results={}
for n in range(505000,507014):
    if n in ids:
        continue
    name,date,cty,addr,txt = get_dn_page(n)
    if name == '':
        continue
    results[n] = [name,date,cty,addr,txt]
    time.sleep(0.05)

505609
505899 2022-08-24 00:00:00 Chris  Ayres (née Kelly) Ballasport, Hill-of-Down,  Meath Westmeath
505900 2022-08-24 00:00:00 Ger (The Piper)  RYAN Newcastle,  Dublin Kildare
505901 2022-08-22 00:00:00 Angela  Buckley (née Barry) Kent Terrace, Castlelyons,  Cork 
505902 2022-08-25 00:00:00 Vinny  Healy Ballindrehid, Ballyhaunis,  Mayo Mayo
505903 2022-08-24 00:00:00 Elizabeth (Betty)  DOYLE (née Haughton) Dublin 8,  Dublin Dublin
505904 2022-08-23 00:00:00 Mary Ellen (Minella)  SEERY (née Sherlock) Greystones,  Wicklow / Athy, Kildare Dublin
505905 2022-08-25 00:00:00 James (Jimmy)  BROHAN Donnycarney,  Dublin Dublin
505906 2022-08-24 00:00:00 John V  Bourke Beaumont,  Dublin Dublin
505907 2022-08-24 00:00:00 Margaret  Concannon (née Flynn) Renmore Road, Renmore,  Galway / Aran Islands, Galway Galway
505908 2022-08-24 00:00:00 Debora (Dora)  Walsh (née Bradford) Rathorig, Rathass, Tralee,  Kerry Kerry
505909 2022-08-25 00:00:00 Susan  McGreevy No 1, Elaghmore, West End, Bundoran,  D

In [23]:
res = pd.DataFrame.from_dict(results,orient='index',columns=['name','date','county','address','notice']).reset_index()
res = res.rename(columns={'index':'id'})

/tmp/ipykernel_3549188/3295222312.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  res = pd.DataFrame.from_dict(results,orient='index',columns=['name','date','county','address','notice']).reset_index()


In [24]:
res

,id,name,date,county,address,notice
0,505899,Chris Ayres (née Kelly),2022-08-24,Westmeath,"Ballasport, Hill-of-Down, Meath","Chris Ayres (nee Kelly), Ballasport, Hill of D..."
1,505900,Ger (The Piper) RYAN,2022-08-24,Kildare,"Newcastle, Dublin","Ryan (Newcastle, Co. Dublin) – Aug 24, 2022, (..."
2,505901,Angela Buckley (née Barry),2022-08-22,,"Kent Terrace, Castlelyons, Cork","Buckley, (née Barry), Boston, Massachusetts & ..."
3,505902,Vinny Healy,2022-08-25,Mayo,"Ballindrehid, Ballyhaunis, Mayo","The death has occurred of Vinny Healy, Ballind..."
4,505903,Elizabeth (Betty) DOYLE (née Haughton),2022-08-24,Dublin,"Dublin 8, Dublin","DOYLE, Elizabeth (Betty) (née Haughton) (Lomba..."
...,...,...,...,...,...,...
1100,507009,Charles EARLEY,2022-09-04,Dublin,"Glasnevin, Dublin / Leitrim","EARLEY Charles (Late of Glasnevin, Dublin 9, f..."
1101,507010,John P (Jack) O'Sullivan,2022-09-05,Kerry,"Killaboona, The Glen, Ballinskelligs, Kerry","John P (Jack) O'Sullivan, Killaboona, The Glen..."
1102,507011,Constantino (Tino) Del-Rio,2022-09-02,Wexford,"Abbeyleix, Laois / Sutton, Dublin","Late of Sutton, Co. Dublin.;2nd September, 202..."
1103,507012,Francis (Frank) Mulvany,2022-09-04,Dublin,"Finglas East, Dublin","Mulvany, Francis (Frank), Finglas East, 4th Se..."


In [25]:
new = pd.concat([df,res]).reset_index(drop=True)
new=new[~new.id.duplicated(keep='first')]
print (len(df),len(res),len(new))
new.to_pickle('rip_dn_scrape.pkl')

505609 1105 506714


In [26]:
#x.to_csv('rip_dn_scrape.csv')

## clean data

In [27]:
x=new
print (len(x))
x=x.replace('',None).dropna(axis=0,subset=['date'])
x['date'] = pd.to_datetime(x['date']).apply(lambda x: x.strftime('%d/%m/%Y'))
x=x.drop_duplicates(['name','notice'])
x=x.drop_duplicates(['name','address'])
x=x.drop_duplicates(['name','date','county'])
x = x[~x.address.isnull()]
nc = ['Fermanagh','Armagh','Tyrone','Down','Antrim','Derry']
x = x[~x.county.isin(nc)]
x = x[~x.address.str.contains('|'.join(nc))]
x=x.sort_values('id')
print (len(x))
#x.to_csv('rip_dn_scrape_processed.csv')
x.to_pickle('rip_dn_scrape_processed.pkl')

506714
471111
